In [ ]:
import os

import geopandas as gpd
import pandas as pd
import shapely.geometry as sg

from aerial_image_detection.raster_utils import RasterData
from aerial_image_detection.sahi_inference import SAHIInferenceModel

RD_crs = "EPSG:28992"

classes_to_keep = [9, 10]  # ["large vehicle", "small vehicle"]

full_images_folder = "../datasets/experiments/parkeren/luchtfotos/luchtfotos_centrum_2025/"
output_folder = "../datasets/experiments/parkeren/detections/"

In [ ]:
sahi_model = SAHIInferenceModel(
    yolo_model_weights_path="../datasets/experiments/parkeren/model_weights/yolo11m-obb.pt",
    confidenc_treshold=0.01,
    image_size=1024,
    slice_height=1024,
    slice_width=1024,
    classes_to_keep=classes_to_keep,
    class_agnostic=True
)

In [ ]:
images = sorted([file for file in os.listdir(full_images_folder) if file.endswith(".tif")])

os.makedirs(output_folder, exist_ok=True)

detections_gdf_list = []

In [ ]:
for i, image_file in enumerate(images):
    print(f"Processing file {i+1}/{len(images)} : {image_file}")

    full_image_path = os.path.join(full_images_folder, image_file)
    raster_data = RasterData(full_image_path)

    sahi_result = sahi_model.predict(
        image=raster_data.as_rgb_img(),
    )

    sahi_predictions = sahi_model.get_prediction_data()

    sahi_predictions["geometry"] = (
        gpd.GeoSeries(
            data=[sg.Polygon(coords) for coords in sahi_predictions["bounding_box"]]
        )
        .affine_transform(raster_data.get_shapely_transform())
    )

    detections_gdf = gpd.GeoDataFrame(
        data=sahi_predictions,
        crs=RD_crs,
    )

    detections_gdf.insert(0, column="source_file", value=image_file)

    output_file = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}.geojson")
    detections_gdf.to_file(output_file, driver='GeoJSON')

    print(f"Predictions written to {output_file}")
    print("-----")

    detections_gdf_list.append(detections_gdf)

In [ ]:
full_detections_gdf = pd.concat(detections_gdf_list)
full_detections_gdf.reset_index(inplace=True, drop=True)
output_file = os.path.join(output_folder, "combined_detections.geojson")
full_detections_gdf.to_file(output_file, driver='GeoJSON', index=True)

In [ ]:
centrum = gpd.read_file("../datasets/experiments/parkeren/GBD_centrum_RD.json")
centrum_detections_gdf = full_detections_gdf[full_detections_gdf.intersects(centrum.geometry[0])]
output_file = os.path.join(output_folder, "combined_detections_centrum.geojson")
centrum_detections_gdf.to_file(output_file, driver='GeoJSON', index=True)

In [ ]:
# Read back from file
centrum_detections_gdf = gpd.read_file("local_data/combined_detections_centrum.geojson").set_index("index")